In [1]:
from atproto import Client
import os

client = Client()
profile = client.login(os.getenv('USERNAME'), os.getenv('PASSWORD'))

In [2]:
cursor = None
databs_posts = []

while True:
    fetched = posts = client.app.bsky.feed.search_posts(params={'q': '#databs', 'cursor': cursor})
    databs_posts = databs_posts + fetched.posts

    if not fetched.cursor:
        break

    cursor = fetched.cursor

In [3]:
def get_all_follows(author):
    cursor = None
    follows = []
    while True:
        fetched = client.app.bsky.graph.get_follows(params={'actor': author, 'cursor': cursor})
        follows = follows + fetched.follows
        if not fetched.cursor:
            break
        cursor = fetched.cursor
    return follows

In [5]:
from tqdm import tqdm

with open('databs.csv', 'w') as f:
    f.write("source,target,source_avatar_url,target_avatar_url\n")
    for p in tqdm(databs_posts):
        source = p.author.handle
        source_avatar_url = p.author.avatar
        author_follows = get_all_follows(source)
        for follow in author_follows:
            f.write(f"{source},{follow.handle},{source_avatar_url},{follow.avatar}\n")

100%|██████████| 50/50 [02:22<00:00,  2.85s/it]
